In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# create a simple feedforward neural net with 1 hidden layer
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# instantiate two models
model1 = Net()
model2 = Net()

In [4]:
dict(model1.named_parameters())

{'fc1.weight': Parameter containing:
 tensor([[-0.5390, -0.1353],
         [-0.5097, -0.5405]], requires_grad=True),
 'fc1.bias': Parameter containing:
 tensor([-0.1775,  0.2634], requires_grad=True),
 'fc2.weight': Parameter containing:
 tensor([[-0.5788,  0.2535]], requires_grad=True),
 'fc2.bias': Parameter containing:
 tensor([-0.4919], requires_grad=True)}

In [5]:
# instantiate the combined model
combined_model = Net()

In [6]:
alpha = nn.Parameter(torch.randn(1))
beta = nn.Parameter(torch.randn(1))

In [7]:
param_names = [name for name, _ in combined_model.named_parameters()]

for name in param_names:
    print(f'replacing {name}')
    # del eval(f'combined_model.{name}')
    exec(f'del combined_model.{name}')
    # x = alpha * eval(f'model1.{name}') + beta * eval(f'model2.{name}')
    exec(f'combined_model.{name} = alpha * model1.{name} + beta * model2.{name}')

replacing fc1.weight
replacing fc1.bias
replacing fc2.weight
replacing fc2.bias


In [8]:
for name, param in model1.named_parameters():
    param.requires_grad = False

for name, param in model2.named_parameters():
    param.requires_grad = False

In [9]:
dict(combined_model.named_parameters())

{}

In [10]:
x = torch.tensor([1., 1.])
y = combined_model(x)
y.backward()

print(f'alpha.grad = {alpha.grad}')
print(f'beta.grad = {beta.grad}')

alpha.grad = tensor([-0.4012])
beta.grad = tensor([0.1048])
